In [1]:
#Importar librerias
import boto3, botocore, json, sys
print("boto3:", boto3.__version__)
print("botocore:", botocore.__version__)


boto3: 1.39.4
botocore: 1.39.4


In [4]:
#probar conexion y si hay tablas existentes
try:
    # 1️Conexión a DynamoDB Local (puerto 8000)
    dynamodb = boto3.client(
        "dynamodb",
        endpoint_url="http://localhost:8000",
        region_name="us-west-2",
        aws_access_key_id="fake",
        aws_secret_access_key="fake",
    )

    # 2️ Llamada de prueba: listar tablas
    tablas = dynamodb.list_tables()["TableNames"]

    # 3️ Confirmación visual
    print("\x1b[32m✓ Conexión exitosa a DynamoDB Local\x1b[0m")
    print(f"Tablas existentes: {tablas if tablas else '— ninguna tabla—'}")

except (BotoCoreError, ClientError) as e:
    print("\x1b[31m✗ Error de conexión\x1b[0m")
    print(e)

✓ Conexión exitosa a DynamoDB Local
Tablas existentes: — ninguna tabla—


In [6]:
# Creacion de  un "resource" 
dynamodb_resource = boto3.resource(
    "dynamodb",
    endpoint_url=dynamodb.meta.endpoint_url,
    region_name=dynamodb.meta.region_name,
    aws_access_key_id=dynamodb._request_signer._credentials.access_key,
    aws_secret_access_key=dynamodb._request_signer._credentials.secret_key,
)

In [7]:
#Crear tabla de pedidos
table_name = "Pedidos"

# Si existe, la borramos para empezar limpio
try:
    dynamodb.delete_table(TableName=table_name)
    dynamodb.get_waiter("table_not_exists").wait(TableName=table_name)
except dynamodb.exceptions.ResourceNotFoundException:
    pass

# Crear la tabla
dynamodb.create_table(
    TableName=table_name,
    AttributeDefinitions=[
        {"AttributeName": "cliente_id", "AttributeType": "S"},
        {"AttributeName": "pedido_id", "AttributeType": "S"},
    ],
    KeySchema=[
        {"AttributeName": "cliente_id", "KeyType": "HASH"},
        {"AttributeName": "pedido_id", "KeyType": "RANGE"},
    ],
    BillingMode="PAY_PER_REQUEST",
)
dynamodb_resource.Table(table_name).wait_until_exists()
print("Tabla Pedidos lista")

Tabla Pedidos lista


In [8]:
from decimal import Decimal
import datetime as dt

In [9]:
#insertar datos de prueba
pedidos = [
    {
        "cliente_id": "C001",
        "pedido_id": "P1001",
        "fecha": dt.date(2025, 7, 15).isoformat(),
        "productos": ["Laptop", "Mouse"],
        "total": Decimal("1299.90"),
    },
    {
        "cliente_id": "C001",
        "pedido_id": "P1002",
        "fecha": dt.date(2025, 7, 16).isoformat(),
        "productos": ["Teclado"],
        "total": Decimal("99.90"),
    },
    {
        "cliente_id": "C002",
        "pedido_id": "P2001",
        "fecha": dt.date(2025, 7, 17).isoformat(),
        "productos": ["Monitor", "HDMI Cable"],
        "total": Decimal("399.90"),
    },
]

table = dynamodb_resource.Table("Pedidos")
for p in pedidos:
    table.put_item(Item=p)

print("Pedidos insertados")

Pedidos insertados


In [10]:
#ver pedidos por cliente
from boto3.dynamodb.conditions import Key

def pedidos_por_cliente(cliente_id):
    resp = table.query(
        KeyConditionExpression=Key("cliente_id").eq(cliente_id)
    )
    return resp["Items"]

print(pedidos_por_cliente("C001"))

[{'fecha': '2025-07-15', 'total': Decimal('1299.9'), 'cliente_id': 'C001', 'pedido_id': 'P1001', 'productos': ['Laptop', 'Mouse']}, {'fecha': '2025-07-16', 'total': Decimal('99.9'), 'cliente_id': 'C001', 'pedido_id': 'P1002', 'productos': ['Teclado']}]


In [11]:
#ver productos de un pedido
def productos_de_pedido(cliente_id, pedido_id):
    resp = table.get_item(Key={"cliente_id": cliente_id, "pedido_id": pedido_id})
    return resp["Item"]["productos"] if "Item" in resp else []

print(productos_de_pedido("C001", "P1001"))

['Laptop', 'Mouse']


In [13]:
#Contador de pedidos por cliente
from collections import Counter

scan = table.scan()
contador = Counter(item["cliente_id"] for item in scan["Items"])
print(dict(contador))

{'C002': 1, 'C001': 2}
